# 미니 프로젝트 - 부동산 아파트 실거래 가격 수집

| 항목명(영문)    | 항목명(국문)    | 크기  | 구분 | 데이터    | 항목설명                                                                               |
| ---------- | ---------- | --- | -- | ------ | ---------------------------------------------------------------------------------- |
| serviceKey | 인증키        | 100 | 1  | 인증키    | 공공데이터포털에서 발급받은 인증키                                                                 |
| pageNo     | 페이지번호      | 4   | 0  | 1      | 페이지번호                                                                              |
| numOfRows  | 한 페이지 결과 수 | 4   | 0  | 10     | 한 페이지 결과 수                                                                         |
| LAWD_CD    | 지역코드       | 5   | 1  | 11110  | [각 지역별 코드 행정표준코드관리시스템(www.code.go.kr)의 법정동코드 10자리 중 앞 5자리](http://www.code.go.kr/) |
| DEAL_YMD   | 계약월        | 6   | 1  | 201512 | 실거래 자료의 계약년월(6자리)                                                                  |

✅ 공공 데이터 포탈 : https://www.data.go.kr/ <br>
✅ PublicDataReader : https://github.com/WooilJeong/PublicDataReader

## 패키지 설치하기 

In [15]:
# XML을 JSON으로 변환해 주는 모듈 
!pip install xmltodict

In [5]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import xmltodict
import datetime

⭐️ URL 및 Columns 지정

In [21]:
# 아파트 매매 URL 및 Columns 
url = "http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev"
columns = ['지역코드', '도로명', '법정동', '지번', '아파트', '건축년도', '층', '전용면적', '년', '월', '일', '거래금액', '도로명건물본번호코드', '도로명건물부번호코드', '도로명시군구코드', '도로명일련번호코드', '도로명지상지하코드', '도로명코드', '법정동본번코드', '법정동부번코드', '법정동시군구코드', '법정동읍면동코드', '법정동지번코드', '일련번호', '거래유형', '중개사소재지', '해제사유발생일', '해제여부']

## 데이터 저장하기 - 단일연월 데이터 조회하기 

⭐️ 파라미터 값 설정 세팅 - 단일월 조회 
1. 인증키 : default parameter
2. 조회할 행수 : defult parameter
3. 시군구 코드 : default parameter
4. 조회할 연월 : input value
    

In [119]:
# 용산구, 2021년도 3월 
service_key = '서비스키'
sigungu_code = '11170'
year_month = '202103'

params = {
    "serviceKey": requests.utils.unquote(service_key),
    "numOfRows": "99999",
    "LAWD_CD": sigungu_code,
    "DEAL_YMD" : year_month 
}
   

⭐️ 빈 데이터 프레임 생성


In [120]:
df = pd.DataFrame(columns=columns)

⭐️ Request 보내기 및 status 확인 

In [121]:
# response 함수에 url, 딕션너리 형태 파라미터, SSL 인증을 생략하는 verify=False 입력 
response = requests.get(url, params=params, verify=False)
response.status_code

200

In [122]:
response

<Response [200]>

⭐️ xmltodict 모듈 사용해서 json형태로 변환 

In [123]:
res_json = xmltodict.parse(response.text)

⭐️ 부동산 매매 데이터 items에 저장

In [125]:
items = res_json['response']['body']["items"]
data = items['item']


In [126]:
sub = pd.DataFrame(data)

In [127]:
df = pd.concat([df,sub], axis=0, ignore_index=True)

In [129]:
df

,지역코드,도로명,법정동,지번,아파트,건축년도,층,전용면적,년,월,...,법정동본번코드,법정동부번코드,법정동시군구코드,법정동읍면동코드,법정동지번코드,일련번호,거래유형,중개사소재지,해제사유발생일,해제여부
0,11170,후암로44길,후암동,1-20,후암맨션,1974,6,72.13,2021,3,...,0001,0020,11170,10100,1,11170-3,None,None,None,None
1,11170,원효로,원효로1가,46-3,용산파크뷰(46-3),2018,15,29.67,2021,3,...,0046,0003,11170,11200,1,11170-3174,None,None,None,None
2,11170,효창원로15길,신창동,102,세방리버하이빌,2005,5,59.76,2021,3,...,0102,0000,11170,11400,1,11170-259,None,None,None,None
3,11170,새창로8길,산천동,204,한강타운,2001,14,59.69,2021,3,...,0204,0000,11170,11500,1,11170-139,None,None,None,None
4,11170,삼개로,청암동,181,GS자이,2005,5,157.43,2021,3,...,0181,0000,11170,11600,1,11170-164,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,11170,독서당로,한남동,828,한남아이파크애비뉴,2017,11,49.6243,2021,3,...,0828,0000,11170,13100,1,11170-3107,None,None,None,None
83,11170,한남대로,한남동,72-1,한남동리첸시아,2004,11,37.53,2021,3,...,0072,0001,11170,13100,1,11170-184,None,None,None,None
84,11170,독서당로,한남동,810,한남더힐,2011,8,59.686,2021,3,...,0810,0000,11170,13100,1,11170-2839,None,None,None,None
85,11170,한남대로,한남동,72-1,한남동리첸시아,2004,14,51.93,2021,3,...,0072,0001,11170,13100,1,11170-184,None,None,None,None


## 데이터 저장하기 - 연월 기간 지정 후 데이터 수집하기 

⭐️ 파라미터 값 설정 세팅 - 연월 기간 지정 후 데이터 수집하기 
1. 인증키 : default parameter
2. 조회할 행수 : defult parameter
3. 시군구 코드 : default parameter
    

In [1]:
# 연월 시점 설정 
# start_year_month : 'yyyymm' or yyyymm
# end_year_month : 'yyyymm' or yyyymm 

⭐️ 서비스키, 시군구 코드, params 값에 설정

In [30]:
# 용산구, 2022년도 5월부터 8월까지 
service_key = '서비스키'
sigungu_code = '11170'

params = {
    "serviceKey": requests.utils.unquote(service_key),
    "numOfRows": "99999",
    "LAWD_CD": sigungu_code,
}

⭐️ 지정된 기간 연월 데이터 값을 추출하는 함수 

In [24]:
def date_list_create (start_year_month, end_year_month) :
    start_date = datetime.datetime.strptime(str(start_year_month), "%Y%m")
    start_date = datetime.datetime.strftime(start_date, "%Y-%m")
    end_date = datetime.datetime.strptime(str(end_year_month), "%Y%m")
    end_date += datetime.timedelta(days=31)
    end_date = datetime.datetime.strftime(end_date, "%Y-%m")
    ts = pd.date_range(start=start_date, end=end_date, freq='m')
    date_list = list(ts.strftime("%Y%m"))
    return date_list

In [25]:
# 2022년 5월부터 2022년 8월까지 날짜 리스트 만들기 
start_year_month = 202205
end_year_month = 202208
date_list = date_list_create(start_year_month, end_year_month)

In [26]:
date_list

['202205', '202206', '202207', '202208']

In [27]:
df = pd.DataFrame(columns=columns)

for year_month in date_list :
    params["DEAL_YMD"] = year_month
    response = requests.get(url, params=params, verify=False)
    res_json = xmltodict.parse(response.text)
    data = res_json['response']['body']["items"]['item']
    sub = pd.DataFrame(data)
    df = pd.concat([df,sub], axis=0, ignore_index=True)

In [28]:
df

,지역코드,도로명,법정동,지번,아파트,건축년도,층,전용면적,년,월,...,법정동본번코드,법정동부번코드,법정동시군구코드,법정동읍면동코드,법정동지번코드,일련번호,거래유형,중개사소재지,해제사유발생일,해제여부
0,11170,후암로28마길,후암동,302,신주에지앙,2004,3,111.4,2022,5,...,0302,0000,11170,10100,1,11170-258,중개거래,서울 용산구,None,None
1,11170,한강대로,동자동,45,센트레빌아스테리움서울,2013,12,131.05,2022,5,...,0045,0000,11170,10700,1,11170-2917,중개거래,서울 용산구,None,None
2,11170,원효로90길,원효로1가,41,용산더프라임,2014,9,142.901,2022,5,...,0041,0000,11170,11200,1,11170-2960,중개거래,서울 용산구,None,None
3,11170,원효로90길,원효로1가,41,용산더프라임,2014,32,142.901,2022,5,...,0041,0000,11170,11200,1,11170-2960,직거래,None,None,None
4,11170,백범로,원효로1가,133-3,리첸시아용산,2010,24,198.32,2022,5,...,0133,0003,11170,11200,1,11170-2811,중개거래,서울 강남구,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,11170,이촌로2가길,이촌동,395,대림,1994,8,84.78,2022,8,...,0395,0000,11170,12900,1,11170-38,중개거래,서울 용산구,None,None
99,11170,이촌로34길,이촌동,400,현대한강,1997,22,84.88,2022,8,...,0400,0000,11170,12900,1,11170-70,중개거래,서울 용산구,None,None
100,11170,이촌로,이촌동,411,동아그린,1999,5,43.02,2022,8,...,0411,0000,11170,12900,1,11170-41,중개거래,서울 용산구,None,None
101,11170,이촌로14길,이촌동,434,동원베네스트,2005,11,84.973,2022,8,...,0434,0000,11170,12900,1,11170-2310,직거래,None,None,None
